## Preprocessing notebook, remove bad points

In [1]:
import sys
import os

# Get the absolute path of parent folder
current_dir = os.path.abspath("")
parent_dir = os.path.join(current_dir, os.pardir)

# Add to sys.path
sys.path.append(parent_dir)

In [2]:
import ml_combat as ml
from ml_combat import data
from ml_combat.MetaModel import MetaModel

#### Imports

In [3]:
from prophet import Prophet
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import plotly.express as px
from statsmodels.tsa.seasonal import seasonal_decompose


#### Body

In [44]:
# get csv_files in actual_submissions
csv_files = [file for file in os.listdir(ml.module_dir + '/../actual_submissions')]
csv_files += [file for file in next(os.walk(ml.module_dir))[1] if file != '__pycache__']

In [45]:
csv_files

['CatComposite_20models_jacob_samp_weight_new_features.csv',
 'xgbc_simen_X_gluonjacob_catcomp_11_8_2023.csv',
 'average_of_lt150_fixed.csv',
 'THIS_GluonW_HPO_and_tuning_data_and_sample_weighting.csv',
 'JacobGluon_duplicat.csv',
 'CatComposite_30models.csv',
 'CatComposite_20models_tuned.csv',
 'average_of_lt150_exclude_avg.csv',
 'CompositeComposite_XGBComp_GluonNoDate_Cat_runJacob.csv',
 'JacobGluon_duplicat_good_quality.csv',
 'JacobGluon_w_sample_imp_10min_select_features.csv',
 'CatComposite_20models_jacob_samp_weight_new_features_retest.csv',
 'JacobGluon_duplicat_good_quality_feat_spes.csv',
 'JacobGluon_duplicat3.csv',
 'JacobGluon_duplicat2.csv',
 'average_of_lt150.csv',
 'JacobGluon_duplicat_0.05.csv',
 'SIMENCatComposite_20models_rstate23-42.csv',
 'JacobGluon_w_sample_imp_3min_saturday.csv',
 'average_of_simen20models_and_143_606701.csv',
 'HENRIKCatComposite_x20_lr0.03_iters8000.csv',
 'CompositeComposite_GluonNoDate_Cat_run_remove_gt9lt4_Jacob.csv',
 'JACOBCatComposite_

In [109]:
sub_dict = {
    # 'xgbc_simen_X_gluonjacob_catcomp_11_8_2023.csv': 144.39969,
    # 'average_of_lt150_fixed.csv': 144.08816,
    # 'CatComposite_30models.csv': 144.596307,
    # 'CatComposite_20models_tuned.csv': 144.516284,
    # 'average_of_lt150_exclude_avg.csv': 143.952789,
    # 'CompositeComposite_XGBComp_GluonNoDate_Cat_runJacob.csv': 143.606701,
    # 'average_of_lt150.csv': 143.317402,
    # 'CompositeComposite_GluonNoDate_Cat_run_remove_gt9lt4_Jacob.csv': 144.570953,
    # 'top_10_only_avg.csv': 142.99779,
    # 'JacobGluon_40min_CatComp_x50.csv': 144.818924,
    # # 'JACOBCatComposite_20models_jacob_samp_weight.csv': 1,
    # # 'HENRIKCatComposite_x20_lr0.03_iters8000.csv': 2,
    # # 'SIMENCatComposite_20models_rstate23-42.csv': 3,
    # 'average_of_simen20models_and_143_606701.csv': 143.214879,
    # 'JacobGluon_w_sample_imp_10min_select_features.csv': 146.191341,
    # 'CatComposite_20models_jacob_samp_weight_new_features.csv': 142.939667,
    # 'CatComposite_20models_jacob_samp_weight_new_features_retest.csv': 10,
    'THIS_CatComposite_20models_jacob_samp_weight_new_features.csv': 1,
    'THIS_GluonW_HPO_and_tuning_data_and_sample_weighting.csv': 2,
    'JacobGluon_duplicat.csv': 5,
    'JacobGluon_w_sample_imp_3min_saturday.csv': 6,
    'JacobGluon_duplicat2.csv': 7,
    'JacobGluon_duplicat3.csv': 8,
    'JacobGluon_duplicat_0.05.csv': 9,
    'JacobGluon_duplicat_bad_split.csv': 10,
    'JacobGluon_duplicat_good_quality.csv': 11,
    'JacobGluon_duplicat_good_quality_feat_spes.csv': 12,
    'JacobGluon_duplicat_good_quality_142.csv': 13,
    'GluonW_HPO_and_tuning_data_and_sample_weighting_and_3levelstack.csv': 50,
    'JacobGluon_duplicat_good_quality_auto_42.csv': 20, # denne er nå feil
    'REPRODUCED_GluonW_HPO_and_tuning_data_and_sample_weighting.csv': 100,
    'CatComposite_new_features_x10_lr0.03_iters8000_nothing_random.csv': 150,
    'REPRODUCED2_GluonW_HPO_and_tuning_data_and_sample_weighting.csv': 101,
    'REPRODUCED_GluonW_HPO_and_tuning_data_and_sample_weighting_no_winter.csv': 102
    }

# sub_dict = { # fyll på med alle
#     'XGBoostComposite_laptop-30.csv': 151.15368,
#     'GluonW_HPO_and_tuning_data.csv': 150.50763,
#     'XGBoostComposite-150.csv': 153.74262,
#     'XGBoostComposite-10.csv': 153.89125,
#     'GluonW_HPO_and_tuning_data_and_sample_weighting.csv': 145.20614,
#     'CompositeComposite_XGBComp_Gluon_Cat_run.csv': 147.27314,
#     'XGBoostComposite_laptop-60.csv': 150.56387,
#     'XGBoostComposite_metalearner_linreg.csv': 153.50807,
#     'XGBoostComposite_metalearner2_linreg.csv': 152.26096
#     }

In [110]:
df = ml.data.get_testing_flattened()
ret = pd.DataFrame()

for csv in sub_dict:
    temp_df = df.copy().merge(pd.read_csv(ml.module_dir + '/../actual_submissions/' + csv), left_index=True, right_index=True)
    temp_df.drop(columns=['id'], inplace=True)
    temp_df.rename(columns={'prediction': 'y_pred'}, inplace=True)
    temp_df = temp_df[['location', 'ds', 'y_pred']]
    temp_df['csv'] = csv
    temp_df['kaggle'] = sub_dict[csv]

    ret = pd.concat([ret, temp_df])

In [111]:
for csv in ret.csv.unique():
    ret.loc[ret.csv == csv, 'best_dev'] = ret[ret.csv == csv]['y_pred'] - ret[ret.kaggle == 2]['y_pred']

In [113]:
for csv in ret.csv.unique():
    print(csv,ret[ret.csv == csv].best_dev.abs().mean() )


THIS_CatComposite_20models_jacob_samp_weight_new_features.csv 50.27788846942168
THIS_GluonW_HPO_and_tuning_data_and_sample_weighting.csv 0.0
JacobGluon_duplicat.csv 32.06368500881798
JacobGluon_w_sample_imp_3min_saturday.csv 44.4423301270077
JacobGluon_duplicat2.csv 19.92429022114242
JacobGluon_duplicat3.csv 18.310927505333883
JacobGluon_duplicat_0.05.csv 28.209519578601537
JacobGluon_duplicat_bad_split.csv 140.95167507680583
JacobGluon_duplicat_good_quality.csv 17.92746108072183
JacobGluon_duplicat_good_quality_feat_spes.csv 17.449028274043453
JacobGluon_duplicat_good_quality_142.csv 24.649826929674656
GluonW_HPO_and_tuning_data_and_sample_weighting_and_3levelstack.csv 16.545928900334058
JacobGluon_duplicat_good_quality_auto_42.csv 17.315305751973774
REPRODUCED_GluonW_HPO_and_tuning_data_and_sample_weighting.csv 19.749431582095074
CatComposite_new_features_x10_lr0.03_iters8000_nothing_random.csv 53.8824706058901
REPRODUCED2_GluonW_HPO_and_tuning_data_and_sample_weighting.csv 14.938031

In [114]:
fig = px.scatter(ret.reset_index(), x="index", y="best_dev", color="kaggle",
                title="Automatic Labels Based on Data Frame Column Names")
fig.show()

In [115]:
fig = px.line(ret.reset_index(), x="index", y="y_pred", color="kaggle",
                title="Automatic Labels Based on Data Frame Column Names")
fig.show()

In [93]:
tp = (ret.groupby(['location','ds']).max().y_pred - ret.groupby(['location','ds']).min().y_pred).reset_index()

In [ ]:
# ret.groupby(['location', 'ds']).mean().reset_index().reset_index()[['index', 'y_pred']].rename(columns={'index':'id', 'y_pred': 'prediction'}).to_csv('average_of_lt155.csv', index=False)

/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_25738/3411814484.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [94]:
df1 = ret[(ret.kaggle == 8)].copy()
df2 = ret[(ret.kaggle == 150)].copy()
df1.y_pred = df1.y_pred*0.5 + df2.y_pred*0.5
df1.groupby(['location', 'ds']).mean().reset_index().reset_index()[['index', 'y_pred']].rename(columns={'index':'id', 'y_pred': 'prediction'}).to_csv('average_of_middleCat_and_middleAutogluon.csv', index=False)

/var/folders/rn/zrxftcb56xg254wsfbhrtbt80000gn/T/ipykernel_93317/3570947399.py:4: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



In [30]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [33]:
t = TabularPredictor.load("./AutogluonModels/ag-20231106_203924")

In [43]:
t.info()

{'path': './AutogluonModels/ag-20231106_203924/',
 'label': 'y',
 'random_state': 0,
 'version': '0.8.2',
 'features': ['weather_data_type',
  'absolute_humidity_2m:gm3',
  'air_density_2m:kgm3',
  'ceiling_height_agl:m',
  'clear_sky_energy_1h:J',
  'clear_sky_rad:W',
  'cloud_base_agl:m',
  'dew_or_rime:idx',
  'dew_point_2m:K',
  'diffuse_rad:W',
  'diffuse_rad_1h:J',
  'direct_rad:W',
  'direct_rad_1h:J',
  'effective_cloud_cover:p',
  'elevation:m',
  'fresh_snow_12h:cm',
  'fresh_snow_1h:cm',
  'fresh_snow_24h:cm',
  'fresh_snow_3h:cm',
  'fresh_snow_6h:cm',
  'is_day:idx',
  'is_in_shadow:idx',
  'msl_pressure:hPa',
  'precip_5min:mm',
  'precip_type_5min:idx',
  'pressure_100m:hPa',
  'pressure_50m:hPa',
  'prob_rime:p',
  'rain_water:kgm2',
  'relative_humidity_1000hPa:p',
  'sfc_pressure:hPa',
  'snow_density:kgm3',
  'snow_depth:cm',
  'snow_melt_10min:mm',
  'snow_water:kgm2',
  'sun_azimuth:d',
  'sun_elevation:d',
  'super_cooled_liquid_water:kgm2',
  't_1000hPa:K',
  'to

In [104]:
sinfo = t.info()#.keys()

In [105]:
sinfo['best_model']

'ExtraTreesMSE_BAG_L2_FULL'

In [106]:
sinfo.keys()

dict_keys(['path', 'label', 'random_state', 'version', 'features', 'feature_metadata_in', 'time_fit_preprocessing', 'time_fit_training', 'time_fit_total', 'time_limit', 'time_train_start', 'num_rows_train', 'num_cols_train', 'num_rows_val', 'num_classes', 'problem_type', 'eval_metric', 'best_model', 'best_model_score_val', 'best_model_stack_level', 'num_models_trained', 'num_bag_folds', 'max_stack_level', 'max_core_stack_level', 'model_info'])

In [108]:
sinfo['model_info']['ExtraTreesMSE_BAG_L2_FULL']

{'name': 'ExtraTreesMSE_BAG_L2_FULL',
 'model_type': 'StackerEnsembleModel',
 'problem_type': 'regression',
 'eval_metric': 'mean_absolute_error',
 'stopping_metric': 'mean_absolute_error',
 'fit_time': 4.901132106781006,
 'num_classes': None,
 'quantile_levels': None,
 'predict_time': None,
 'val_score': None,
 'hyperparameters': {'use_orig_features': True,
  'max_base_models': 25,
  'max_base_models_per_type': 5,
  'save_bag_folds': True,
  'use_child_oof': True},
 'hyperparameters_fit': {},
 'hyperparameters_nondefault': ['use_child_oof', 'save_bag_folds'],
 'ag_args_fit': {'max_memory_usage_ratio': 1.15,
  'max_time_limit_ratio': 1.0,
  'max_time_limit': None,
  'min_time_limit': 0,
  'valid_raw_types': None,
  'valid_special_types': None,
  'ignored_type_group_special': None,
  'ignored_type_group_raw': None,
  'get_features_kwargs': None,
  'get_features_kwargs_extra': None,
  'predict_1_batch_size': None,
  'temperature_scalar': None,
  'drop_unique': False},
 'num_features': 58